In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]
ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1]/sum(mpc_bus[j][1] for j in range(118))
print(ratio_bus)
print(sum(ratio_bus[i] for i in range(118)))
print(sum(mpc_bus[j][1] for j in range(118)))

topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) 
unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit', header=0) 
reserve_up = 150
reserve_down = 150
nodes = 118

[0.01202263 0.00471476 0.00919378 0.00919378 0.         0.01225837
 0.00447902 0.00660066 0.         0.         0.01650165 0.01107968
 0.00801509 0.00330033 0.02121641 0.00589345 0.00259312 0.01414427
 0.0106082  0.00424328 0.00330033 0.00235738 0.00165017 0.00306459
 0.         0.         0.01673739 0.00400754 0.00565771 0.
 0.01013673 0.01390853 0.00542197 0.01390853 0.00777935 0.00730787
 0.         0.         0.00636492 0.0155587  0.0087223  0.02263083
 0.00424328 0.00377181 0.01249411 0.00660066 0.00801509 0.00471476
 0.02050919 0.00400754 0.00400754 0.00424328 0.00542197 0.02663838
 0.01485149 0.01980198 0.00282885 0.00282885 0.06529939 0.01838755
 0.         0.01815182 0.         0.         0.         0.00919378
 0.00660066 0.         0.         0.0155587  0.         0.00282885
 0.00141443 0.01603017 0.01107968 0.01603017 0.01438001 0.01673739
 0.00919378 0.03064592 0.         0.01272984 0.00471476 0.00259312
 0.00565771 0.0049505  0.         0.01131542 0.         0.03842527
 0.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import os


def predict_hourly_y(hour, X_test, theta, beta):
    
    y_pred = np.dot(X_test, theta) + beta
    return y_pred


def combine_and_sort_predictions(predictions, trend_values):
    
    all_predictions = np.concatenate(predictions)
    
    all_trends = np.concatenate(trend_values)
    
    sorted_indices = np.argsort(all_trends)
    
    sorted_predictions = all_predictions[sorted_indices]
    return sorted_predictions


def train_and_predict():
    
    all_predictions = []
    
    all_trends = []
    
    if not os.path.exists('./hourly_predictions'):
        os.makedirs('./hourly_predictions')
    
    lambda_reg = 1e4  
    
    for hour in range(1, 25):
        
        X_train_file = f'./train_splits/X_train_hour_{hour}.csv'
        Y_train_file = f'./train_splits/Y_train_hour_{hour}.csv'
        X_test_file = f'./test_splits/X_test_hour_{hour}.csv'
        Y_test_file = f'./test_splits/Y_test_hour_{hour}.csv'
        X_train = pd.read_csv(X_train_file)
        Y_train = pd.read_csv(Y_train_file)
        X_test = pd.read_csv(X_test_file)
        Y_test = pd.read_csv(Y_test_file)
        
        model = gp.Model()
        P_c = model.addVars(unit.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='P_c') 
        P_l = model.addVars(X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='P_w') 
        R_U = model.addVars(unit.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='R_U') 
        R_D = model.addVars(unit.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='R_D')
        r_U = model.addVars(unit.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='r_U') 
        r_D = model.addVars(unit.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='r_D') 
        P_lsh = model.addVars(topo.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='P_lsh') 
        theta_pl = model.addVars(X_train.shape[1], lb=-1000, vtype=GRB.CONTINUOUS, name='theta_pl') 
        delta_DA = model.addVars(topo.shape[0], X_train.shape[0], lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_DA')
        delta_RT = model.addVars(topo.shape[0], X_train.shape[0], lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_RT')
        ss = model.addVars(topo.shape[0], X_train.shape[0], lb=0, vtype=GRB.CONTINUOUS, name='ss') 
        b = model.addVar(lb=-10000, vtype=GRB.CONTINUOUS, name='b')
        c = 9000 
        c_s = 1750 
        
        model.setObjective(gp.quicksum(R_U[i,j]*unit.iloc[i,5] + R_D[i,j]*unit.iloc[i,6] + P_c[i,j]*unit.iloc[i,2] 
                        + (r_U[i,j]-r_D[i,j])*unit.iloc[i,2] for i in range(unit.shape[0]) for j in range(X_train.shape[0]))
                        + gp.quicksum(P_lsh[i,j] * c for i in range(topo.shape[0]) for j in range(X_train.shape[0]))
                        + gp.quicksum(ss[i,j]*c_s for i in range(topo.shape[0]) for j in range(X_train.shape[0]))
                        + lambda_reg * gp.quicksum(theta_pl[r] * theta_pl[r] for r in range(X_train.shape[1]))) 
        for k in range(X_train.shape[0]):
            
            model.addConstr(gp.quicksum(R_U[i,k] for i in range(unit.shape[0])) == reserve_up)
            model.addConstr(gp.quicksum(R_D[i,k] for i in range(unit.shape[0])) == reserve_down)
                
            for i in range(unit.shape[0]):
                model.addConstr(R_U[i,k] <= unit.iloc[i,3])
                model.addConstr(R_D[i,k] <= unit.iloc[i,4])
                
            for t in range(topo.shape[0]):
                    model.addConstr(P_c[t,k]-P_l[k]*ratio_bus[t]
                    -gp.quicksum(topo.iloc[t,j]*(delta_DA[t,k]-delta_DA[j,k]) for j in range(topo.shape[0])) == 0)       
            model.addConstr(delta_DA[0,k] == 0)  
                
            for i in range(unit.shape[0]):
                model.addConstr(P_c[i,k] <= unit.iloc[i,1]-R_U[i,k])
                model.addConstr(P_c[i,k] >= R_D[i,k])
                
            
            model.addConstr(P_l[k] == gp.quicksum(theta_pl[r] * X_train.iloc[k,r] for r in range(X_train.shape[1])) + b)
            model.addConstr(P_l[k] >= 0)
                
            for t in range(topo.shape[0]):
                model.addConstr(r_U[t,k]-r_D[t,k] - ss[t,k] - Y_train.iloc[k,0]*ratio_bus[t] + P_l[k]*ratio_bus[t] + P_lsh[t,k]
                -gp.quicksum(topo.iloc[t,j]*(delta_RT[t,k]-delta_RT[j,k]) - topo.iloc[t,j]*(delta_DA[t,k]-delta_DA[j,k]) for j in range(topo.shape[0]))==0)
                
            for i in range(unit.shape[0]):
                model.addConstr(r_U[i,k] <= R_U[i,k])
                model.addConstr(r_D[i,k] <= R_D[i,k])
            model.addConstr(delta_RT[0,k] == 0)  
                
            for t in range(topo.shape[0]):
                model.addConstr(P_lsh[t, k] >= 0)  
                
            for i in range(nodes):
                for j in range(nodes):
                    if i != j and topo.iloc[i,j] < -0.1:
                        model.addConstr(topo.iloc[i,j] * (delta_DA[i,k] - delta_DA[j,k]) <= 175)
                        model.addConstr(topo.iloc[i,j] * (delta_DA[i,k] - delta_DA[j,k]) >= -175)
                
            for i in range(nodes):
                for j in range(nodes):
                    if i != j and topo.iloc[i,j] < -0.1:
                        model.addConstr(topo.iloc[i,j] * (delta_RT[i,k] - delta_RT[j,k]) <= 175)
                        model.addConstr(topo.iloc[i,j] * (delta_RT[i,k] - delta_RT[j,k]) >= -175)
        model.optimize()

        obj_power = gp.quicksum(R_U[i,j]*unit.iloc[i,5] + R_D[i,j]*unit.iloc[i,6] + P_c[i,j]*unit.iloc[i,2] 
                        + (r_U[i,j]-r_D[i,j])*unit.iloc[i,2] for i in range(unit.shape[0]) for j in range(X_train.shape[0])).getValue()
        obj_load_shedding = gp.quicksum(P_lsh[i,j] * c for i in range(topo.shape[0]) for j in range(X_train.shape[0])).getValue()
        obj_ss = gp.quicksum(ss[i,j]*c_s for i in range(topo.shape[0]) for j in range(X_train.shape[0])).getValue()
        obj_regularization = lambda_reg * gp.quicksum(theta_pl[r] * theta_pl[r] for r in range(X_train.shape[1])).getValue()

        
        theta = np.array([theta_pl[i].X for i in range(X_train.shape[1])])
        beta = b.X
        
        y_pred = predict_hourly_y(hour, X_test, theta, beta)
        
        all_predictions.append(y_pred)
        
        all_trends.append(X_test['trend'].values)
        
        hourly_predictions_df = pd.DataFrame({'Predicted_Y': y_pred, 'trend': X_test['trend'].values})
        hourly_predictions_df.to_csv(f'./hourly_predictions/hour_{hour}_predictions.csv', index=False)
    
    sorted_predictions = combine_and_sort_predictions(all_predictions, all_trends)
    
    sorted_predictions_df = pd.DataFrame(sorted_predictions, columns=['Predicted_Y'])
    sorted_predictions_df.to_csv('prediction_GF.csv', index=False)


train_and_predict()

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2025-03-30
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13400F, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 172500 rows, 73362 columns and 457677 nonzeros
Model fingerprint: 0x03047d27
Model has 14 quadratic objective terms
Coefficient statistics:
  Matrix range     [5e-04, 2e+03]
  Objective range  [4e+01, 9e+03]
  QObjective range [2e+04, 2e+04]
  Bounds range     [2e+02, 1e+04]
  RHS range        [4e-01, 8e+02]
Presolve removed 120819 rows and 26151 columns
Presolve time: 0.22s
Presolved: 51681 rows, 71430 columns, 257646 nonzeros
Presolved model has 14 quadratic objective terms
Ordering time: 0.17s

Barrier statistics:
 Dense cols : 69
 AA' NZ   